## 本例是将V0.2版本的分析结果中，去除晚上02:00-07:55（开始时间）的数据

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
def getEstTripsCount(df,n=5):  
    trips_count = df["TRIPS_COUNT"]
    estTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if i < n-1:
            estTripsCount.append(0)
        else:
            estTripsCount.append(sum(trips_count[i+1-n:i+1]))
    return estTripsCount

def getRealTripsCount(df,n=5):
    trips_count = df["TRIPS_COUNT"]
    realTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if l-i-1 < n:
            realTripsCount.append(0)
        else:
            realTripsCount.append(sum(trips_count[i+1:i+1+n]))
    return realTripsCount

In [3]:
data = pd.read_csv("from_06_01_5_minute_stat.csv")
data["FROM_TIME"] = pd.to_datetime(data["FROM_TIME"])

In [26]:
def getEstAndRealTime(dataFrame, carStorageTime, disPersion):
    """
    返回预付发车量大于空车差量的开始时间列表和实际发车量大于空车差量的时间列表
    """
    carStorageCount = int(carStorageTime/5)
    # 计算空车差量
    dataFrame["VEHICLE_COUNT_dispersion"] = data.VEHICLE_COUNT*(1+disPersion)
    # 计算预估发车量
    estTripsCount = getEstTripsCount(dataFrame, n=carStorageCount)
    # 计算实际发车量
    realTripsCount = getRealTripsCount(dataFrame, n=carStorageCount)
    
    trips_count_dict = {
        "FROM_TIME":dataFrame["FROM_TIME"],
        "estTripsCount":estTripsCount,
        "realTripsCount":realTripsCount
    }
    # 合并数据集
    df = dataFrame.merge(pd.DataFrame(trips_count_dict), on="FROM_TIME")
    csv_name = "calc_{}_{}.csv".format(carStorageTime,disPersion)
   
    
    df["est_lg"]="1" if df["estTripsCount"]>df["VEHICLE_COUNT_dispersion"] else "0"
    df["real_lg"] ="1" if df["realTripsCount"]>df["VEHICLE_COUNT_dispersion"] else "0"                                                      
    df.to_csv(csv_name)
    df["HOUR"] = df["FROM_TIME"].dt.hour
    
    
    # 筛选出需要的记录 1.预估大于空车差量的结果 2.真实大于预估差量的结果 3.预估和真实均大于差量的结果
    df_est_lg = df[(df["estTripsCount"]>df["VEHICLE_COUNT_dispersion"]) & (df["HOUR"].isin({0,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23}))]
    df_real_lg = df[(df["realTripsCount"]>df["VEHICLE_COUNT_dispersion"]) & (df["HOUR"].isin({0,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23}))]
    # 获取筛选结果的时间列表，供返回
    df_est_lg_fromtime = df_est_lg["FROM_TIME"]
    df_real_lg_fromtime = df_real_lg["FROM_TIME"]
    return df_est_lg_fromtime, df_real_lg_fromtime   


In [27]:
def calcRate(df, carStorageTime, disPersion):
    est_time, real_time = getEstAndRealTime(df, carStorageTime, disPersion)
    est_time = set(est_time)
    real_time = set(real_time)
    
    accuracy_rate = len(est_time & real_time)/len(est_time)
    false_positive_rate = len(est_time-real_time)/len(est_time)
    return accuracy_rate, false_positive_rate

In [28]:
data.index

RangeIndex(start=0, stop=5601, step=1)

In [29]:
accuracy_rate, false_positive_rate = calcRate(data, 20, 0.05)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [8]:
print(accuracy_rate, false_positive_rate)

0.4666666666666667 0.5333333333333333


In [9]:
carStorageTimes = [20,25,30,35,40]
disPersion = [0.05,0.1,0.15]

result = {
    "carStorageTimes":[],
    "disPersion":[],
    "accuracy_rate":[],
    "false_positive_rate":[]
}

for cst in carStorageTimes:
    for dp in disPersion:
        accuracy_rate, false_positive_rate = calcRate(data, cst, dp)
        result["carStorageTimes"].append(cst)
        result["disPersion"].append(dp)
        result["accuracy_rate"].append(accuracy_rate)
        result["false_positive_rate"].append(false_positive_rate)
        
df_result = pd.DataFrame(result)
df_result

,carStorageTimes,disPersion,accuracy_rate,false_positive_rate
0,20,0.05,0.466667,0.533333
1,20,0.10,0.435897,0.564103
2,20,0.15,0.369231,0.630769
3,25,0.05,0.513944,0.486056
4,25,0.10,0.502463,0.497537
5,25,0.15,0.487342,0.512658
6,30,0.05,0.604374,0.395626
7,30,0.10,0.588235,0.411765
8,30,0.15,0.557746,0.442254
9,35,0.05,0.722721,0.277279
